# Check gpu:

In [1]:
!nvidia-smi

Sun Mar 14 12:43:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 461.72       Driver Version: 461.72       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108... WDDM  | 00000000:07:00.0  On |                  N/A |
| 41%   56C    P0    64W / 250W |    717MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Clone the project

In [2]:
%cd /content/
!git clone https://github.com/Hubert482/CAIN.git

[WinError 2] Nie można odnaleźć określonego pliku: '/content/'
C:\Users\Hubert\Documents\GitHub\CAIN


Cloning into 'CAIN'...


# connect to gdrive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Install requirements

In [4]:
pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 26kB/s 
     |████████████████████████████████| 6.7MB 49.7MB/s 
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.5.1+cu101 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:
      Successfully uninstalled torchvision-0.9.0+cu101


# generate  DataSet


In [ ]:
#@markdown Folder path with all the videos
inputPath = "/content/gdrive/MyDrive/Dataset_videos/" #@param{type:"string"}
#@markdown Path to generate the Dataset
outputPath = "/content/CAIN/data/vimeo_triplet/" #@param{type:"string"}
#@markdown The bigger the value, the more alike the frames need to be to be considered a triplet.
psnr = 25 #@param{type:"number"}
global  outputPath
#@markdown resize to 448x256
resize = True #@param{type:"boolean"}

#Original dataset information:
#Vimeo-90K triplets dataset contains 91701 triplets extracted from 15k video clips.
#Each triplet is a short RGB video sequence that consists of 3 frames with fixed resolution 448x256


def calculate_psnr(img1, img2, max_value=255):
    """"Calculating peak signal-to-noise ratio (PSNR) between two images."""
    i1 = PIL.Image.open(img1).convert('RGB')
    i2 = PIL.Image.open(img2).convert('RGB')

    mse = np.mean((np.array(i1, dtype=np.float32) - np.array(i2, dtype=np.float32)) ** 2)
    if mse == 0:
        return 100
    return 20 * np.log10(max_value / (np.sqrt(mse)))


def IsDiffScenes(img1, img2, diff=25):
  return calculate_psnr(img1, img2) <= diff


import os
import shutil
from shutil import copyfile
import time
import PIL
import numpy as np


if not os.path.isdir(outputPath):
  os.makedirs(outputPath)

if not os.path.isdir(outputPath + "sequences"):
  os.makedirs(outputPath + "sequences")

counter = 1
for video in sorted(os.listdir(inputPath)):
  temp = "/content/.temp/"
  path = inputPath + video
  if os.path.isdir(temp):
    shutil.rmtree(temp)
    #time.sleep(2)
  os.makedirs(temp)

  rescale = ""
  if resize:
    rescale="scale=448:256,"
  !ffmpeg -i "{path}" -vf "{rescale}mpdecimate" -vsync 0 -qscale:v 1 -pix_fmt rgb24  "{temp}%10d.png"

  root =outputPath + "/sequences/" + str(counter).zfill(3) + "/"
  if not os.path.isdir(root):
    os.makedirs(root)


  triIndex = 1
  images = sorted(os.listdir(temp))
  for i in range(2, len(images)) :
    im1 = temp + images[i-2]
    im2 =  temp + images[i-1]
    im3 =  temp + images[i]
    if IsDiffScenes(im1, im2, psnr) == False and IsDiffScenes(im2, im3, psnr) == False:
      #print("Triplet Found")
      triPath = root + str(triIndex).zfill(5) + "/"
      if not os.path.isdir(triPath):
        os.makedirs(triPath)
      copyfile(im1, triPath + "im1.png")
      copyfile(im2, triPath + "im2.png")
      copyfile(im3, triPath + "im3.png")
      triIndex +=1

  counter += 1

testingPercentage = 0.1 #@param{type:"slider", min:0.1, max:0.9, step:0.1}
database=outputPath
import random

if os.path.isfile(database + "tri_testlist.txt"):
  os.remove(database + "tri_testlist.txt")
if os.path.isfile(database + "tri_trainlist.txt"):
  os.remove(database + "tri_trainlist.txt")

from glob import glob
folders = glob(database + "sequences/*/")

paths = []

for i in range(0, len(folders)):
  triplets = glob(folders[i] + "/*/")
  for ii in range(0, len(triplets)):
    innerF = os.path.basename((os.path.dirname(triplets[ii])))
    outF = os.path.basename((os.path.dirname(folders[i])))
    paths.append(outF + "/" + innerF)

def partitionRankings(rawRatings, testPercent):
    howManyNumbers = int(round(testPercent*len(rawRatings)))
    shuffled = rawRatings[:]
    random.shuffle(shuffled)
    return shuffled[howManyNumbers:], shuffled[:howManyNumbers]

training, test = partitionRankings(paths, testingPercentage)


trainTxt = database + "/tri_trainlist.txt"
trainFile = open(trainTxt, 'w')

testTxt = database + "/tri_testlist.txt"
testFile = open(testTxt, 'w')

for txt in training:
  trainFile.write(txt + "\n")
trainFile.close()

for txt in test:
  testFile.write(txt + "\n")
testFile.close()

print(training)
print(test)


# Train the model (This take a long time)

In [ ]:
%cd /content/CAIN
import os 
#@markdown model is save in your google drive (CAINMODEL/)

dataset = "/content/CAIN/data/vimeo_triplet/" #@param{type:"string"}
exp_name = "exp/" #@param{type:"string"}

resume = False #@param{type:"boolean"}
batch_size =  20 #@param{type:"number"}
n_resblocks =  12 #@param{type:"number"}
n_resgroups =  5 #@param{type:"number"}

if resume==True:
  resume="--resume"

!python main.py --colab --data_root "{dataset}" --batch_size {batch_size} --n_resblocks n_resblocks --n_resgroups {n_resgroups} --test_batch_size {batch_size} {resume} --exp_name "{exp_name}"